This notebook is used to formalize and store the data used in anxiety dettection project into a single HDF style file

we have following 3 data collected for the project
1. treadmill data
2. fNIR data
3. EMG data

For each data, we have 4 trials: N1, N2, P1, P2

The following code will read each data file, parse them, and store then into a single file
The resulting data will have following 12 sub data
1. emg_N1
2. emg_N2
3. emg_P1
4. emg_P2
5. fNIRS_N1
6. fNIRS_N2
7. fNIRS_P1
8. fNIRS_P2
9. TM_N1
10. TM_N2
11. TM_P1
12. TM_P2

In [1]:
trial_number = '302'

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# ML libraries 
from sklearn.cluster import KMeans
from sklearn.neighbors.kde import KernelDensity


# Computation & Signal Processing
from scipy import signal
import numpy as np
import pandas as pd
import pylab as pl
import pickle

#biosppy package for ecg signal analysis
from biosppy import storage
from biosppy.signals import ecg

from utils import * #import data import, clean up and sampling functions. 
# import time
import time
import os
import glob
import time

from ECG_feature_extractor_1000 import *
from utils import * #import data import, clean up and sampling functions. 

In [2]:
emg_ecg_columns = ['X[s]', 'R RECTUS FEMORIS: EMG 1', 'L RECTUS FEMORIS: EMG 2', 'Trigno sensor 3: EMG 3', 
                 'L BICEPS FEMORIS: EMG 4', 'Trigno Mini sensor 5: EMG 5', 'R BICEPS FEMORIS: EMG 6',
                 'R TIBIALIS ANTERIOR: EMG 7','L TIBIALIS ANTERIOR: EMG 8', 'L GASTROCNEMIUS LATERAL HEAD: EMG 11',
                 'L GASTROCNEMIUS MEDIAL HEAD: EMG 12', 'R GASTROCNEMIUS LATERAL HEAD: EMG 13',
                 'R GASTROCNEMIUS MEDIAL HEAD: EMG 14', 'L PECTORALIS MAJOR: EKG 16']

OA data module to store the data used for OA project

In [3]:
class OA_data:
    def __init__(self, number):
        self.data_array = []
        self.number = number
        
        
    def fit_emg(self, data_dir = 'csv/'):
        trial_name_1 = self.number + "_NW1"
        trial_name_2 = self.number + "_NW2"
        trial_name_3 = self.number + "_P1"
        trial_name_4 = self.number + "_P2"
        
        trial_data = glob.glob(data_dir + trial_name_1 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg1 = load_data(trial_name, data_dir = '')
            self.df_emg1 = self.df_emg1[emg_ecg_columns]
            self.df_emg1 = delsys_cleanup(self.df_emg1, column='all')
            self.df_emg1 = self.df_emg1.rename(columns={'X[s]': 'time'})
        except:
            print("EMG file for " + trial_name_1 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_2 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg2 = load_data(trial_name, data_dir = '')
            self.df_emg2 = self.df_emg2[emg_ecg_columns]
            self.df_emg2 = delsys_cleanup(self.df_emg2, column='all')
            self.df_emg2 = self.df_emg2.rename(columns={'X[s]': 'time'})
        except:
            print("EMG file for " + trial_name_2 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_3 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg3 = load_data(trial_name, data_dir = '')
            self.df_emg3 = self.df_emg3[emg_ecg_columns]
            self.df_emg3 = delsys_cleanup(self.df_emg3, column='all')
            self.df_emg3 = self.df_emg3.rename(columns={'X[s]': 'time'})
        except:
            print("EMG file for " + trial_name_3 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_4 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg4 = load_data(trial_name, data_dir = '')
            self.df_emg4 = self.df_emg4[emg_ecg_columns]
            self.df_emg4 = delsys_cleanup(self.df_emg4, column='all')
            self.df_emg4 = self.df_emg4.rename(columns={'X[s]': 'time'})
        except:
            print("EMG file for " + trial_name_4 + " does not exist")
    
    def fit_fNIRS(self, data_dir = 'fNIRS_Walking data'):
        file_name = '/OA_FNIRS_2019_WALK_' + self.number + '_oxydata.txt'
        df_fNIRS = pd.read_csv(data_dir+file_name,sep='\t')
        df_fNIRS = df_fNIRS.rename(columns={'1':'subject_ID', '2':'cohort', '3':'block','4':'trial_i','5':'HbO1','6':'HbR1','7':'HbO2','7':'HbR2','8':'HbO3','9':'HbR3','10':'HbO4','11':'HbR4','12':'HbO5','13':'HbR5','14':'HbO6','15':'HbR6','16':'HbO7','17':'HbR7','18':'HbO8','19':'HbR8','20':'HbO9','21':'HbR9','23':'HbO10','24':'HbR10','25':'HbO11','26':'HbR11','27':'HbO12','28':'HbR12','29':'HbO13','30':'HbR13','31':'HbO14','32':'HbR14','33':'HbO15','34':'HbR15','35':'HbO16','36':'HbR16'})
        df_fNIRS['time'] = [i*0.5 for i in range(len(df_fNIRS))]
        df_fNIRS = df_fNIRS[['time','subject_ID', 'cohort', 'block', 'trial_i', 'HbO1', 'HbR1', 'HbR2',
                            'HbO3', 'HbR3', 'HbO4', 'HbR4', 'HbO5', 'HbR5', 'HbO6', 'HbR6', 'HbO7',
                            'HbR7', 'HbO8', 'HbR8', 'HbO9', 'HbR9', '22', 'HbO10', 'HbR10', 'HbO11',
                            'HbR11', 'HbO12', 'HbR12', 'HbO13', 'HbR13', 'HbO14', 'HbR14', 'HbO15',
                            'HbR15', 'HbO16', 'HbR16']]
        self.df_fNIRS_N1 = df_fNIRS[df_fNIRS['trial_i']==1]
        self.df_fNIRS_N2 = df_fNIRS[df_fNIRS['trial_i']==2]
        self.df_fNIRS_P1 = df_fNIRS[df_fNIRS['trial_i']==3]
        self.df_fNIRS_P2 = df_fNIRS[df_fNIRS['trial_i']==4]
        
    
    def fit_TMdata(self, data_dir = 'Treadmill_data'):
        trial_name_1 = self.number + "_N1"
        trial_name_2 = self.number + "_N2"
        trial_name_3 = self.number + "_P1"
        trial_name_4 = self.number + "_P2"
        
        file_name = '/OA_' + trial_name_1 + '_RAWDATA.csv'
        self.df_tm1 = load_data(file_name, data_dir, header=1)
        self.df_tm1 = self.df_tm1.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_2 + '_RAWDATA.csv'
        self.df_tm2 = load_data(file_name, data_dir, header=1)
        self.df_tm2 = self.df_tm2.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_3 + '_RAWDATA.csv'
        self.df_tm3 = load_data(file_name, data_dir, header=1)
        self.df_tm3 = self.df_tm3.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_4 + '_RAWDATA.csv'
        self.df_tm4 = load_data(file_name, data_dir, header=1)
        self.df_tm4 = self.df_tm4.rename(columns={'Time': 'time'})
        
    def save(self):
        hdf = pd.HDFStore(self.number +'.h5')
        hdf.put('emg_N1', self.df_emg1, format='table', data_columns=True)
        hdf.put('emg_N2', self.df_emg2, format='table', data_columns=True)
        hdf.put('emg_P1', self.df_emg3, format='table', data_columns=True)
        hdf.put('emg_P2', self.df_emg4, format='table', data_columns=True)
        hdf.put('fNIRS_N1', self.df_fNIRS_N1, format='table', data_columns=True)
        hdf.put('fNIRS_N2', self.df_fNIRS_N2, format='table', data_columns=True)
        hdf.put('fNIRS_P1', self.df_fNIRS_P1, format='table', data_columns=True)
        hdf.put('fNIRS_P2', self.df_fNIRS_P2, format='table', data_columns=True)
        hdf.put('TM_N1', self.df_tm1, format='table', data_columns=True)
        hdf.put('TM_N2', self.df_tm2, format='table', data_columns=True)
        hdf.put('TM_P1', self.df_tm3, format='table', data_columns=True)
        hdf.put('TM_P2', self.df_tm4, format='table', data_columns=True)
        hdf.close()

We firs create the data module in order to save the data files

Pass the trial number to initialize the module.

In [4]:
abc = OA_data('302')

For next step, we fit the data module using the corresponding data files

The default directory of each data file is shown below:

1. EMG: csv/
2. fNIRS: fNIRS_Walking data
3. TreadMill: Treadmill_data

If data file are note located at default dir, pass the dir name as an argument to load the data

In [5]:
abc.fit_emg()
abc.fit_fNIRS()
abc.fit_TMdata()

In [6]:
abc.df_emg1

,time,R RECTUS FEMORIS: EMG 1,L RECTUS FEMORIS: EMG 2,Trigno sensor 3: EMG 3,L BICEPS FEMORIS: EMG 4,Trigno Mini sensor 5: EMG 5,R BICEPS FEMORIS: EMG 6,R TIBIALIS ANTERIOR: EMG 7,L TIBIALIS ANTERIOR: EMG 8,L GASTROCNEMIUS LATERAL HEAD: EMG 11,L GASTROCNEMIUS MEDIAL HEAD: EMG 12,R GASTROCNEMIUS LATERAL HEAD: EMG 13,R GASTROCNEMIUS MEDIAL HEAD: EMG 14,L PECTORALIS MAJOR: EKG 16
0,0.001038,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.001558,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.002077,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.002596,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.003115,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577790,300.007400,0.000018,-0.000012,0,-3.054856e-05,0,0.000008,0.000012,0.000028,0.000017,0.000008,-0.000142,-0.000003,0.000156
577791,300.007900,0.000018,-0.000010,0,-2.937362e-05,0,0.000017,0.000017,0.000034,0.000017,0.000006,-0.000060,0.000007,0.000164
577792,300.008400,0.000015,-0.000009,0,-1.980621e-05,0,0.000019,0.000013,0.000018,0.000015,-0.000010,-0.000013,0.000023,0.000171
577793,300.008900,0.000011,-0.000008,0,-7.217517e-06,0,0.000016,0.000009,0.000003,0.000013,-0.000045,0.000018,0.000044,0.000176


Final step

Save the data module using save() function

CAUTION!
- Make sure the same file you are going to save does not exist. If does, make sure to delete it before saving it.

This step takes significant amount of time. (over 5 min)

In [7]:
abc.save()

Following code shows how to retrieve the data back from saved file

In [11]:
#Only importing Pandas is enough for reading the file
#First Argument: file name
#Second Argument: data to retrieve
OA_302_emg_N1 = pd.read_hdf('302.h5','TM_N1')

In [12]:
OA_302_emg_N1

,time,TreadMill_FZ,TreadMill_MX,TreadMill_MY,COPX,COPY,Speed
0,0.000,736.618751,-351.782328,326.673580,0.442870,0.486323,0.295932
1,0.002,736.451355,-351.954637,326.576903,0.442851,0.485321,0.295932
2,0.004,736.203503,-350.282718,326.743444,0.442839,0.484378,0.295932
3,0.006,736.440620,-350.541270,327.176333,0.442833,0.483486,0.295932
4,0.008,736.616666,-351.818457,327.194170,0.442834,0.482646,0.295932
...,...,...,...,...,...,...,...
102114,204.228,731.871226,-358.262855,329.066601,0.449437,0.489191,0.296770
102115,204.230,732.389858,-358.472622,328.953867,0.449440,0.489155,0.296770
102116,204.232,732.327308,-358.693928,329.278858,0.449443,0.489125,0.296770
102117,204.234,NaN,NaN,NaN,NaN,NaN,0.296770


Following code will perform the data synchronization.


In [8]:
#core code to do the time syn chronization
#change the data_dir_sync argument to the 
def cut_plz(trial_name, data_dir_sync = 'Sync_txt_file/OA_302_B_WMP/'):
    #trial_name should be a string in the format of xxx_yy(xxx is number and yy is trial)
    number = trial_name[:3]
    trial = trial_name[-2:]
    data_dir_sync = data_dir_sync + trial
    sync_emg = pd.read_csv(data_dir_sync+'/EMGtime.txt', sep='\s+')
    sync_fnirs = pd.read_csv(data_dir_sync+'/fNIRstime.txt', sep='\s+')
    sync_tm = pd.read_csv(data_dir_sync+'/treadmill_v.txt', sep='\s+')
    
    res = next(x for x, val in enumerate(sync_tm['TreadmillV']) if val > 4.98) 
    
    t_tm = sync_tm['Time'][res]
    t_fnirs = sync_fnirs['RStime'][0]
    t_emg = sync_emg['begintime'][0]
    
    emg = pd.read_hdf(number + ".h5", "emg_" + trial)
    fNIRS = pd.read_hdf(number + ".h5", "fNIRS_" + trial)
    treadmill = pd.read_hdf(number + ".h5", "TM_" + trial)
    
    if t_emg > t_tm and t_emg > t_fnirs: #, cut tm & fnirs

        #treadmill data
        dt = t_emg - t_tm 
        treadmill = cutting(treadmill, dt)

        #fNIRS
        dt = t_emg - t_fnirs
        fNIRS = cutting(fNIRS, dt)
        return emg, treadmill, fNIRS
        

    elif t_fnirs > t_tm and t_fnirs > t_emg:

        #treadmill data
        dt = t_fnirs - t_tm 
        treadmill = cutting(treadmill, dt)

        #emg, acc & ecg  
        dt = t_fnirs - t_emg
        emg = cutting(emg, dt)
        return emg, treadmill, fNIRS

    elif t_tm > t_emg and t_tm > t_fnirs:
        #emg, acc & ecg  
        dt = t_tm - t_emg 
        emg = cutting(emg, dt)

        #fNIRS
        dt = t_tm - t_fnirs
        fNIRS = cutting(fNIRS, dt)
        return emg, treadmill, fNIRS

In [9]:
#wrapper function
def time_synch(number):
    hdf = pd.HDFStore(number +'_time_sync.h5')
    trial = ['N1','N2','P1','P2']
    for i in trial:
        emg, treadmill, fNIRS = cut_plz(number + '_' + i)
        hdf.put('emg_' + i, emg, format='table', data_columns=True)
        hdf.put('fNIRS_' + i, fNIRS, format='table', data_columns=True)
        hdf.put('TM_' + i, treadmill, format='table', data_columns=True)
    hdf.close()

The following code will do the time synchronization
file will be stored in the format "302_time_sync.h5"

In [11]:
time_synch('302')

In [20]:
pd.read_hdf('302.h5','emg_N1')

,time,R RECTUS FEMORIS: EMG 1,L RECTUS FEMORIS: EMG 2,Trigno sensor 3: EMG 3,L BICEPS FEMORIS: EMG 4,Trigno Mini sensor 5: EMG 5,R BICEPS FEMORIS: EMG 6,R TIBIALIS ANTERIOR: EMG 7,L TIBIALIS ANTERIOR: EMG 8,L GASTROCNEMIUS LATERAL HEAD: EMG 11,L GASTROCNEMIUS MEDIAL HEAD: EMG 12,R GASTROCNEMIUS LATERAL HEAD: EMG 13,R GASTROCNEMIUS MEDIAL HEAD: EMG 14,L PECTORALIS MAJOR: EKG 16
0,0.001038,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.001558,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.002077,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.002596,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.003115,0.000000,0.000000,0,0.000000e+00,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577790,300.007400,0.000018,-0.000012,0,-3.054856e-05,0,0.000008,0.000012,0.000028,0.000017,0.000008,-0.000142,-0.000003,0.000156
577791,300.007900,0.000018,-0.000010,0,-2.937362e-05,0,0.000017,0.000017,0.000034,0.000017,0.000006,-0.000060,0.000007,0.000164
577792,300.008400,0.000015,-0.000009,0,-1.980621e-05,0,0.000019,0.000013,0.000018,0.000015,-0.000010,-0.000013,0.000023,0.000171
577793,300.008900,0.000011,-0.000008,0,-7.217517e-06,0,0.000016,0.000009,0.000003,0.000013,-0.000045,0.000018,0.000044,0.000176


In [14]:
pd.read_hdf('302_time_sync.h5','emg_N1')

,time,R RECTUS FEMORIS: EMG 1,L RECTUS FEMORIS: EMG 2,Trigno sensor 3: EMG 3,L BICEPS FEMORIS: EMG 4,Trigno Mini sensor 5: EMG 5,R BICEPS FEMORIS: EMG 6,R TIBIALIS ANTERIOR: EMG 7,L TIBIALIS ANTERIOR: EMG 8,L GASTROCNEMIUS LATERAL HEAD: EMG 11,L GASTROCNEMIUS MEDIAL HEAD: EMG 12,R GASTROCNEMIUS LATERAL HEAD: EMG 13,R GASTROCNEMIUS MEDIAL HEAD: EMG 14,L PECTORALIS MAJOR: EKG 16
0,0.000091,0.000011,-3.860533e-06,0,-2.500954e-05,0,0.000002,0.000011,-0.000004,0.000015,-0.000031,-0.000008,0.000003,-0.000015
1,0.000611,0.000010,-3.021286e-06,0,-2.467384e-05,0,0.000003,0.000012,-0.000003,0.000018,-0.000033,-0.000010,0.000006,-0.000016
2,0.001131,0.000011,-2.349889e-06,0,-2.568093e-05,0,0.000004,0.000011,-0.000003,0.000020,-0.000032,-0.000011,0.000007,-0.000016
3,0.001651,0.000011,-8.392462e-07,0,-2.735943e-05,0,0.000003,0.000011,-0.000002,0.000020,-0.000028,-0.000008,0.000006,-0.000016
4,0.002171,0.000014,1.678492e-07,0,-2.870222e-05,0,0.000004,0.000009,0.000002,0.000019,-0.000025,-0.000005,0.000005,-0.000016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519866,269.930531,0.000018,-1.225299e-05,0,-3.054856e-05,0,0.000008,0.000012,0.000028,0.000017,0.000008,-0.000142,-0.000003,0.000156
519867,269.931031,0.000018,-1.023880e-05,0,-2.937362e-05,0,0.000017,0.000017,0.000034,0.000017,0.000006,-0.000060,0.000007,0.000164
519868,269.931531,0.000015,-8.560311e-06,0,-1.980621e-05,0,0.000019,0.000013,0.000018,0.000015,-0.000010,-0.000013,0.000023,0.000171
519869,269.932031,0.000011,-7.888914e-06,0,-7.217517e-06,0,0.000016,0.000009,0.000003,0.000013,-0.000045,0.000018,0.000044,0.000176


In [15]:
pd.read_hdf('302_time_sync.h5','TM_N1')

,time,TreadMill_FZ,TreadMill_MX,TreadMill_MY,COPX,COPY,Speed
0,0.000,736.618751,-351.782328,326.673580,0.442870,0.486323,0.295932
1,0.002,736.451355,-351.954637,326.576903,0.442851,0.485321,0.295932
2,0.004,736.203503,-350.282718,326.743444,0.442839,0.484378,0.295932
3,0.006,736.440620,-350.541270,327.176333,0.442833,0.483486,0.295932
4,0.008,736.616666,-351.818457,327.194170,0.442834,0.482646,0.295932
...,...,...,...,...,...,...,...
102114,204.228,731.871226,-358.262855,329.066601,0.449437,0.489191,0.296770
102115,204.230,732.389858,-358.472622,328.953867,0.449440,0.489155,0.296770
102116,204.232,732.327308,-358.693928,329.278858,0.449443,0.489125,0.296770
102117,204.234,NaN,NaN,NaN,NaN,NaN,0.296770


In [16]:
pd.read_hdf('302_time_sync.h5','fNIRS_N1')

,time,subject_ID,cohort,block,trial_i,HbO1,HbR1,HbR2,HbO3,HbR3,...,HbO12,HbR12,HbO13,HbR13,HbO14,HbR14,HbO15,HbR15,HbO16,HbR16
0,0.06892,302,3,4,1,-0.4294,0.08035,-1.619,-0.3985,-0.4311,...,-3.015,0.2654,-1.1160,0.6018,-1.908,0.09463,6.48,-4.189,-0.7961,0.4412
1,0.56892,302,3,4,1,-0.4623,0.10320,-1.576,-0.3810,-0.4282,...,-2.956,0.3127,-1.0790,0.5709,-1.879,0.08890,7.018,-3.826,-0.7434,0.4063
2,1.06892,302,3,4,1,-0.4964,0.14920,-1.522,-0.3337,-0.4198,...,-2.862,0.3805,-1.0360,0.5280,-1.833,0.08521,7.83,-4.687,-0.6824,0.3676
3,1.56892,302,3,4,1,-0.5289,0.21390,-1.460,-0.2604,-0.4066,...,-2.744,0.4634,-0.9929,0.4770,-1.775,0.08469,8.911,-6.834,-0.6193,0.3306
4,2.06892,302,3,4,1,-0.5561,0.29030,-1.392,-0.1678,-0.3886,...,-2.613,0.5544,-0.9547,0.4234,-1.710,0.08889,10.14,-9.979,-0.5597,0.3010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,294.56892,302,3,4,1,-2.5490,1.66600,-3.591,1.7190,-1.7310,...,-5.224,1.0990,-4.1090,-0.6825,-5.458,-0.98950,14.39,-52.82,-2.5180,0.1533
590,295.06892,302,3,4,1,-2.4480,1.57900,-3.493,1.6560,-1.6840,...,-5.054,1.0470,-4.0040,-0.5973,-5.292,-0.92310,17.41,-55.62,-2.3480,0.2070
591,295.56892,302,3,4,1,-2.2950,1.45100,-3.340,1.5560,-1.6070,...,-4.891,1.0070,-3.9250,-0.4705,-5.150,-0.82890,20.43,-58.01,-2.2000,0.3005
592,296.06892,302,3,4,1,-2.1060,1.29300,-3.150,1.4260,-1.5090,...,-4.751,0.9830,-3.8820,-0.3060,-5.048,-0.71390,23.45,-59.97,-2.0870,0.4313


In [19]:
pd.read_hdf('302.h5','fNIRS_N1')

,time,subject_ID,cohort,block,trial_i,HbO1,HbR1,HbR2,HbO3,HbR3,...,HbO12,HbR12,HbO13,HbR13,HbO14,HbR14,HbO15,HbR15,HbO16,HbR16
0,0.0,302,3,4,1,0.4448,-0.148700,0.4951,-0.12040,0.2088,...,0.3386,0.2854,0.3441,0.09651,0.1313,0.1924,7.276,-7.491,0.09617,0.2183
1,0.5,302,3,4,1,0.4789,-0.103200,0.5122,-0.07613,0.2308,...,0.4285,0.2520,0.3548,0.08879,0.1912,0.1843,6.254,-5.931,0.11840,0.1981
2,1.0,302,3,4,1,0.5005,-0.060990,0.5221,-0.03719,0.2475,...,0.5005,0.2205,0.3570,0.07543,0.2445,0.1748,5.144,-4.443,0.13770,0.1764
3,1.5,302,3,4,1,0.5093,-0.024520,0.5253,-0.00524,0.2579,...,0.5533,0.1912,0.3513,0.05776,0.2916,0.1637,4.049,-3.117,0.15450,0.1541
4,2.0,302,3,4,1,0.5049,0.004962,0.5213,0.01908,0.2611,...,0.5853,0.1636,0.3375,0.03753,0.3313,0.1505,3.047,-2.014,0.16790,0.1320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,362.0,302,3,4,1,-2.5490,1.666000,-3.5910,1.71900,-1.7310,...,-5.2240,1.0990,-4.1090,-0.68250,-5.4580,-0.9895,14.39,-52.82,-2.51800,0.1533
725,362.5,302,3,4,1,-2.4480,1.579000,-3.4930,1.65600,-1.6840,...,-5.0540,1.0470,-4.0040,-0.59730,-5.2920,-0.9231,17.41,-55.62,-2.34800,0.2070
726,363.0,302,3,4,1,-2.2950,1.451000,-3.3400,1.55600,-1.6070,...,-4.8910,1.0070,-3.9250,-0.47050,-5.1500,-0.8289,20.43,-58.01,-2.20000,0.3005
727,363.5,302,3,4,1,-2.1060,1.293000,-3.1500,1.42600,-1.5090,...,-4.7510,0.9830,-3.8820,-0.30600,-5.0480,-0.7139,23.45,-59.97,-2.08700,0.4313


In [18]:
data_dir_sync = 'Sync_txt_file/OA_302_B_WMP/N1'
sync_emg = pd.read_csv(data_dir_sync+'/EMGtime.txt', sep='\s+')
sync_fnirs = pd.read_csv(data_dir_sync+'/fNIRstime.txt', sep='\s+')
sync_tm = pd.read_csv(data_dir_sync+'/treadmill_v.txt', sep='\s+')
    
res = next(x for x, val in enumerate(sync_tm['TreadmillV']) if val > 4.98) 
    
t_tm = sync_tm['Time'][res]
t_fnirs = sync_fnirs['RStime'][0]
t_emg = sync_emg['begintime'][0]
print("t_tm: " + str(t_tm))
print("t_fnirs: " + str(t_fnirs))
print("t_emg: " + str(t_emg))

t_tm: 96.2733982896
t_fnirs: 28.842317896799997
t_emg: 66.1965297877
